# Edge Detection

Author(s): Raj Magesh Gauthaman (rgautha1@jh.edu)

Reference: [Konishi et al. (2003)](https://doi.org/10.1109/TPAMI.2003.1159946)

---

This notebook implements an edge detection algorithm.

## Utilities

In [ ]:
from __future__ import annotations
from collections.abc import Collection
import itertools

import numpy as np
import numpy.typing as npt
import torch
import torch.nn as nn
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.stats import gaussian_kde
from PIL import Image


def plot_image(image: npt.NDArray, **kwargs) -> mpl.figure.Figure:
    """Plot an image."""
    fig, ax = plt.subplots()
    ax.axis("off")
    ax.imshow(image, cmap="gray", **kwargs)
    fig.show()


def load_data(id: int = 0) -> tuple[np.ndarray, np.ndarray]:
    """id can be from 0 to 6 (inclusive)."""
    edge_map = np.array(Image.open(f"data/edge_detection/edge_maps/{id}.bmp").convert("1"))
    image = np.array(Image.open(f"data/edge_detection/images/{id}.jpg").convert("L")).astype(float)
    image /= 255
    return image, edge_map


def compute_gradient(image: npt.NDArray[float]) -> npt.NDArray[float]:
    dx, dy = np.gradient(image)
    return np.sqrt(dx ** 2 + dy ** 2)

## Algorithm

### Loading the data

Let's load an image and its hand-annotated edge map.

In [ ]:
image, ground_truth = load_data(2)

fig, axes = plt.subplots(nrows=1, ncols=2)
axes[0].imshow(image, cmap="gray")
axes[0].set_title("image")
axes[1].imshow(ground_truth, cmap="gray")
axes[1].set_title("ground-truth edge map")

for ax in axes.flat:
    ax.axis("off")

fig.tight_layout()
fig.show()

### Effect of smoothing

We compute the gradient of the image (i) directly and (ii) after smoothing with a Gaussian filter, and visualize the magnitude of the gradient.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
axes[0].imshow(compute_gradient(image), cmap="gray")
axes[0].set_title("$|\\nabla I|$")
axes[1].imshow(compute_gradient(gaussian_filter(image, sigma=2)), cmap="gray")
axes[1].set_title("$|\\nabla (G \star I)|$")

for ax in axes.flat:
    ax.axis("off")
    
fig.tight_layout()
fig.show()

### Computing likelihoods

Here, we compute the distribution of the gradient magnitudes for edge and non-edge pixels.

In [ ]:
def find_gradient_on_edge(gradient: npt.NDArray[float], edge_map: npt.NDArray[bool]) -> npt.NDArray[float]:
    indices = np.nonzero(edge_map.flatten())

    # allow for inaccurate boundary labelling by finding edge pixel in a 3x3 neighborhood
    [offx, offy] = np.meshgrid(np.arange(-1, 2), np.arange(-1, 2))
    offx = offx.flatten()
    offy = offy.flatten()

    gradient_copy = np.copy(gradient)
    for i in range(9):
        im = np.roll(gradient, offx[i], axis=1) # x axis
        im = np.roll(im, offy[i], axis=0) # y axis    
        gradient_copy = np.maximum(gradient_copy, im)

    return gradient_copy.flatten()[indices]


def find_gradient_off_edge(gradient: npt.NDArray[float], edge_map: npt.NDArray[bool]) -> npt.NDArray[float]:
    indices = np.nonzero(~edge_map)
    return gradient[indices] 


gradient = compute_gradient(image)

on = find_gradient_on_edge(gradient, ground_truth)
kde_on = gaussian_kde(on, bw_method=0.01 / on.std(ddof=1))
off = find_gradient_off_edge(gradient, ground_truth)
kde_off = gaussian_kde(off, bw_method=0.01 / off.std(ddof=1))

bins = np.linspace(0, 0.5, 100)

fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True)
axes[0].hist(on, bins=bins, label="edge pixels", alpha=0.75, density=True)
axes[0].hist(off, bins=bins, label=r"non-edge pixels", alpha=0.75, density=True)
axes[0].set_ylabel("normalized histogram")

axes[1].plot(bins, kde_on(bins))
axes[1].plot(bins, kde_off(bins))
axes[1].set_ylabel("kernel density estimate")

fig.legend()
fig.supxlabel("magnitude of gradient ($|\\nabla I |$)")
fig.tight_layout()
fig.show()

### Classification

Now, we can classify pixels as edge pixels or non-edge pixels. Note: this cell will take a long time (~10 min) to run.

In [ ]:
p_on = kde_on(gradient.flatten()).reshape(gradient.shape)
p_off = kde_off(gradient.flatten()).reshape(gradient.shape)

threshold = 1
decision = np.log(p_on / p_off) > threshold

fig, axes = plt.subplots(ncols=3, figsize=(8, 4))
axes[0].imshow(p_on, cmap="gray")
axes[0].set_title("P(edge)")
axes[1].imshow(p_off, cmap="gray")
axes[1].set_title("P(non-edge | gradient)")
axes[2].imshow(decision, cmap="gray")
axes[2].set_title(f"decision (threshold = {threshold})")

for ax in axes.flat:
    ax.axis("off")
fig.tight_layout()

### ROC curve

Now that we can classify pixels as either edge pixels or non-edge pixels, we can compute an ROC curve by varying the decision threshold.

In [ ]:
def compute_roc(criterion: npt.NDArray[float], ground_truth: npt.NDArray[bool], thresholds: Collection[float]) -> tuple[npt.NDArray[float], npt.NDArray[float]]:
    prediction = np.stack([criterion >= threshold for threshold in thresholds])

    true_positive = prediction & ground_truth
    true_negative = ~prediction & ~ground_truth

    false_positive = prediction & ~ground_truth
    false_negative = ~prediction & ground_truth

    true_positive_rate = true_positive.sum(axis=(-2, -1)) / ground_truth.sum()
    false_positive_rate = false_positive.sum(axis=(-2, -1)) / (~ground_truth).sum()
    return true_positive_rate, false_positive_rate


criterion = np.log(p_on / p_off)
thresholds = np.arange(-5, 5, step=0.1)
tpr, fpr = compute_roc(criterion, ground_truth=ground_truth, thresholds=thresholds)

fig, ax = plt.subplots()

points = np.array([fpr, tpr]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

norm = plt.Normalize(thresholds.min(), thresholds.max())
lc = mpl.collections.LineCollection(segments=segments, cmap="turbo_r", norm=norm, linewidth=4)
lc.set_array(thresholds)
line = ax.add_collection(lc)
ax.set_xlabel("false positive rate")
ax.set_ylabel("true positive rate")
ax.axis("square")
ax.set_xlim(left=0, right=1)
ax.set_ylim(bottom=0, top=1)
ax.set_title("receiver operating characteristic (ROC) curve")
cb = fig.colorbar(line, ax=ax, label="threshold")
fig.show()

## Questions

### Q1 (6 points)

Load another image and apply this edge detection algorithm. Find a good threshold and display your result.

### Q2 (8 points)

Repeat Q1 using the image gradient after smoothing the image (i.e. $\nabla G \star I$ instead of $\nabla I$). Show your results for a couple of different variances ($\sigma^2$) of the Gaussian filter.